In [143]:
# Advent of Code 2022
# https://adventofcode.com/2022/day/7

In [144]:
with open("input.txt") as f:
    input = f.read()

    lines = input.split('\n')
    # remove blank lines
    lines = list(filter(None, lines))




In [145]:
##########
# Directory Tree Structure: Dictionary with links to parent and child items
##########
 
# Directory tree is a dictionary of dictionaries.  
# Each item in the dictionary is either a file or a directory.
# Format for each item is as follows:
# {<id>:{
#   'item_id':<id>, 
#   'item':<'dir or 'file'>, 
#   'parent_id':<id of the parent directory>,
#   'name':<name of the file or directory>,
#   'contents':[<list of ids for the contents of this directory if the item is a directory>],
#   'size':<file size if the item is a file>,
#   'depth': <how many levels down from the root is this item?>}
# }

# This structure lets you trace any item in the tree to its parent directory and its contents.


In [146]:
##########
# Functions to interact with the tree
##########

In [147]:
# set up a new tree. Clears previous tree if there was one.
def reset_tree():
    
    global tree
    global wd
    global next_id

    tree = {}

    '''set up the tree with a root directory,
    # id = 1, parent directory id = 0 (0 is an arbitrary id since the root 
    # directory has no actual parent directory.)'''
    tree = {0:{
        'item_id':0, 
        'item':'dir', 
        'parent_id':0, 
        'name':'root', 
        'contents':[], 
        'size':0, 
        'depth':0},
        1:{
        'item_id':1, 
        'item':'dir', 
        'parent_id':0, 
        'name':'root', 
        'contents':[], 
        'size':0, 
        'depth':0}}

    # set root as the default working directory to start with
    wd = tree[1]

    # set next available ID to assign 
    next_id = 2


In [148]:
# list the names of all items contained in a directory
def contents_names(directory):
    names = []
    for c in directory['contents']:
        names.append(tree[c]['name'])
    return names


In [149]:
def create_subdirectory(subdir_name):
    global next_id
    global wd
    tree[next_id]={
        'item_id':next_id, 
        'item':'dir', 
        'parent_id':wd['item_id'], 
        'name':subdir_name, 
        'contents':[], 
        'size':0, 
        'depth':wd['depth']+1}
    # add subdirectory to list of items in working directory
    wd['contents'].append(next_id)
    # update next available id since this one is taken now
    next_id += 1


In [150]:
def create_file(file_name, size):
    global next_id
    global wd
    tree[next_id]={
        'item_id':next_id, 
        'item':'file', 
        'parent_id':wd['item_id'], 
        'name':file_name, 
        'contents':[], 
        'size':size, 
        'depth':wd['depth']+1}
    # add file to list of items in working directory
    wd['contents'].append(next_id)
    next_id +=1
    

In [151]:
def move_to_subdirectory(subdir_name):
    global wd
    for c in wd['contents']:
        if tree[c]['name'] == subdir_name:
            wd = tree[c]
            

In [152]:
def move_to_parent():
    global wd
    parent_id = wd['parent_id']
    parent_dir = tree[parent_id]
    wd = parent_dir
    

In [153]:
def cd(l):
    ''' reads in an instruction line (l) and executes CD instructions: 
    either moves to the specified directory or creates it if it doesn't
    exist yet and then moves to it.'''

    global wd
    global next_id

    subdir_name = l[5:]
    
    # cd \  -- go to root directory
    if subdir_name == '\\':
        
        # working directory = root directory
        wd = tree[1]

    # cd ..  -- move up one directory level
    elif subdir_name == '..':
        # change working directory to parent directory
        move_to_parent()
    
    # cd <directry name>  -- move down one level to specified subdirectory
    else:             
       
        # if subdirectory already exists
        if subdir_name in contents_names(wd):
            move_to_subdirectory(subdir_name)
                
        # if subdirectory doesn't exist:
        else:
            create_subdirectory(subdir_name)
            move_to_subdirectory(subdir_name)


In [155]:
##########
# Acting on LS lines
##########

# no action needed. LS just indicates the lines that follow
# will require action (adding them to the tree if they don't)
# already exist.

In [156]:
# Acting on listed directories

def ls_dir(l):
    '''Reads in an instruction line (l) and treats it
    as a directory listed under an $ ls command. Creates 
    the directory if it doesn't exist yet. Does not move
    to the subdirectory.'''

    global wd
    global next_id   
            
    subdir_name = l[4:]

    # if subdirectory doesn't exist:
    if subdir_name not in contents_names(wd):
        create_subdirectory(subdir_name)
     

In [157]:
# Acting on listed directories

def ls_file(l):
    '''Reads in an instruction line (l) and treats it
    as a file listed under an $ ls command. Creates 
    the file if it doesn't exist yet.'''

    global wd
    global next_id 

    size, file_name = l.split(' ')
    size = int(size)

    # if file doesn't exist:
    if file_name not in contents_names(wd):
        create_file(file_name, size)


In [158]:
def read_tree(lines):
    '''Reads each line and builds the tree accordingly'''
    global wd
    global next_id
    
    for l in lines:

        if l[:5] == r'$ cd ':
            cd(l)

        elif l == r'$ ls':
            pass
            
        elif l[:4] == r'dir ':
            ls_dir(l)

        else:
            ls_file(l)

In [159]:
# Add the size of an item to its parent directory's total size
def add_size_to_parent(i):
    parent_id = tree[i]['parent_id']
    tree[parent_id]['size'] += tree[i]['size']



In [160]:
##########
# Create a full tree with file and directory sizes
# from the input lines
##########

reset_tree()

read_tree(lines)

# maximum depth in the tree
max_depth = max([tree[i]['depth'] for i in tree])

# list tree depths from highest to lowest
depth_list = [i for i in range(max_depth, 0,-1)]

# loop through items in the tree starting 
# at the bottom (highest depth number).
# Add the size of every item to its parent
# directory size.

for d in depth_list:
    for i in tree:
        if tree[i]['depth'] == d:
            add_size_to_parent(i)



In [161]:
#########
# Find all of the directories with a total size of at most 100000
# and sum them up
##########

answer = 0

for i in tree:
    if tree[i]['item'] == 'dir':
        if tree[i]['size'] <= 100000:
            answer += tree[i]['size']

print(answer)


1141028
